In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import h3
import os
from shapely import wkt
from shapely.geometry import box, Polygon
from folium.plugins import Geocoder
from folium import FeatureGroup, LayerControl

# Get the city from command-line arguments
city = os.environ.get('CITY', 'Default City Name')
base_path = "/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/"

# Sanitize the city name
city_sanitized = city.split(",")[0].replace(" ", "_")
city_path = f"{base_path}{city_sanitized}" 

# Load data
all_lts_df = pd.read_csv(f"{city_path}_all_lts.csv")
all_lts_df['geometry'] = all_lts_df['geometry'].apply(wkt.loads)

# Convert the DataFrame to a GeoDataFrame
all_lts = gpd.GeoDataFrame(all_lts_df, geometry='geometry')
all_lts.crs = "EPSG:32632"
all_lts_projected = all_lts.to_crs(epsg=4326)

gdf_nodes = pd.read_csv(f"{city_path}_gdf_nodes.csv", index_col=0)

color_palette = ["forestgreen", "dodgerblue", "#f4e800", "firebrick", "#000000", "purple"]
lts_classes = [1, 2, 3, 4, "undefined", "not cyclable"]
colors = dict(zip(lts_classes, color_palette))

mean_latitude = all_lts_projected.geometry.apply(lambda geom: geom.centroid.y).mean()
mean_longitude = all_lts_projected.geometry.apply(lambda geom: geom.centroid.x).mean()

map_osm = folium.Map(location=[mean_latitude, mean_longitude], zoom_start=11.5)

geocoder = Geocoder(
    position='topleft',
    control=True,
    collapsed=True,
    auto_type=True,
    placeholder='Search for a location...',
    max_zoom=15,
    hide_marker=True
).add_to(map_osm)

feature_groups = {}

for lts_class, color in colors.items():
    if isinstance(lts_class, str):  # For "Undefined" and "Not cyclable"
        name = lts_class
    else:
        name = f"LTS {lts_class}"
    toggle_switch_html = f'''
    <div class="toggle-switch active" onclick="toggleSwitch(this)" style="background-color: {color};"></div>
    <span>{name}</span>
    '''
    is_visible = False if lts_class == "undefined" else True
    feature_group = FeatureGroup(name=toggle_switch_html, show=is_visible)
    feature_groups[lts_class] = feature_group
    feature_group.add_to(map_osm)

# Function to generate hexagons
def get_hexagons_for_bounds(bounds, resolution):
    bounding_polygon = box(*bounds)
    center = bounding_polygon.centroid
    hex_center = h3.geo_to_h3(center.y, center.x, resolution)
    hexagons = h3.k_ring(hex_center, 100)
    hexagons = [hexagon for hexagon in hexagons if h3.h3_to_geo_boundary(hexagon, geo_json=True)]
    return hexagons

hexagons = get_hexagons_for_bounds(all_lts_projected.total_bounds, 9)
bounding_polygon = box(*all_lts_projected.total_bounds)

# Filter hexagons to keep only those intersecting with the bounding polygon
hexagons = [h for h in hexagons if Polygon(h3.h3_to_geo_boundary(h, geo_json=True)).intersects(bounding_polygon)]

hex_gdf = gpd.GeoDataFrame({'hex_id': hexagons, 
                            'geometry': [Polygon(h3.h3_to_geo_boundary(h, geo_json=True)) for h in hexagons]}, 
                           crs="EPSG:4326")

# Function to find prevalent LTS within each hexagon
def get_prevalent_lts(hex_geometry):
    # Find streets within the hexagon
    streets_within_hex = all_lts_projected[all_lts_projected.intersects(hex_geometry)]
    if streets_within_hex.empty:
        return np.nan
    # Return prevalent LTS
    return streets_within_hex['lts'].value_counts().idxmax()

hex_gdf['lts'] = hex_gdf['geometry'].apply(get_prevalent_lts)

for _, row in hex_gdf.iterrows():
    if pd.isna(row['lts']):
        lts_class = "undefined"
        color = colors[lts_class]
    elif row['lts'] not in feature_groups:
        lts_class = "undefined"
        color = colors[lts_class]
    else:
        streets_within_hex = all_lts_projected[all_lts_projected.intersects(row['geometry'])]
        names = streets_within_hex['name'].fillna("").str.lower()
        if any(names.str.contains(keyword).any() for keyword in ["tangenziale", "superstrada", "strada statale", "autostrada", "strada a scorrimento veloce"]) and not names.str.contains("bici|cicla|ciclo").any():
            lts_class = "not cyclable"
            color = colors[lts_class]
        else:
            lts_class = row['lts']
            color = colors[lts_class]
    
    folium.GeoJson(
        row['geometry'], 
        style_function=lambda _, color=color: {'color': "#000000", 'fillColor': color, 'fillOpacity': 0.7, 'weight': 2}
    ).add_to(feature_groups[lts_class])

LayerControl(position='topright').add_to(map_osm)

map_html = open("custom_map.html", "r").read()
map_osm.get_root().html.add_child(folium.Element(map_html))

map_osm.save(f"{city_path}_choropleth_lts_map.html")

map_osm

FileNotFoundError: [Errno 2] No such file or directory: '/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/Default_City_Name_all_lts.csv'